# Phase 2: CPU-Based Scaling Experiments for LABS Challenge

This notebook implements Phase 2 CPU experiments for the NVIDIA iQuHACK LABS challenge.

**Task:**
- For each sequence length N ∈ {32, 48, 64}
- For 5 random seeds
- Run both random-seeded MTS and quantum-seeded MTS
- Ensure identical parameters (time budget, population size, scoring)
- Log results with: N, seed, init_type, best_energy, runtime_seconds
- Save to `results/phase2_cpu_results.json`

**Important:** This notebook is locked to CPU execution only (no GPU).


In [ ]:
# SPDX-License-Identifier: Apache-2.0 AND CC-BY-NC-4.0

import cudaq
import numpy as np
import json
import time
import os
from math import floor
from collections import deque
from dataclasses import dataclass
import matplotlib.pyplot as plt
import pandas as pd

# Ensure CPU-only execution
cudaq.set_target("qpp-cpu")

# Create results directory if it doesn't exist
os.makedirs("results", exist_ok=True)

print("CUDA-Q target:", cudaq.get_target().name)
print("Starting Phase 2 CPU Experiments...")


## Core LABS Functions

Energy computation and bit conversion utilities from Phase 1.


In [ ]:
def bits01_to_pm1(bits01: np.ndarray) -> np.ndarray:
    """Convert 0/1 bits to -1/+1 spins."""
    return np.where(bits01 > 0, 1, -1).astype(np.int8)

def pm1_to_bits01(spins: np.ndarray) -> np.ndarray:
    """Convert -1/+1 spins to 0/1 bits."""
    return (spins > 0).astype(np.int8)

def labs_energy(spins_pm1: np.ndarray) -> int:
    """
    Compute LABS energy for a spin sequence.
    E(s) = sum_{k=1}^{N-1} C_k^2, where C_k = sum_{i=1}^{N-k} s_i * s_{i+k}
    """
    s = spins_pm1.astype(np.int16)
    n = len(s)
    E = 0
    for k in range(1, n):
        Ck = int(np.dot(s[:-k], s[k:]))
        E += Ck * Ck
    return int(E)


## Memetic Tabu Search (MTS) Implementation

Classical optimizer from Phase 1, with identical configuration parameters.


In [ ]:
def combine_uniform(parent_a: np.ndarray, parent_b: np.ndarray, rng: np.random.Generator) -> np.ndarray:
    """Uniformly combine two parent bitstrings."""
    mask = rng.random(parent_a.shape[0]) < 0.5
    return np.where(mask, parent_a, parent_b).astype(np.int8)

def mutate_flip_bits(spins_pm1: np.ndarray, p_mut: float, rng: np.random.Generator) -> np.ndarray:
    """Mutate bitstring by flipping bits with probability p_mut."""
    flips = rng.random(spins_pm1.shape[0]) < p_mut
    out = spins_pm1.copy()
    out[flips] *= -1
    return out

def best_single_flip(spins: np.ndarray, tabu_set: set):
    """Find the best single-bit flip that is not tabu."""
    current_E = labs_energy(spins)
    best_E = current_E
    best_idx = None
    
    for i in range(len(spins)):
        candidate = spins.copy()
        candidate[i] *= -1
        E = labs_energy(candidate)

        if i not in tabu_set and E < best_E:
            best_E = E
            best_idx = i

    if best_idx is None:
        return spins, current_E, None

    spins[best_idx] *= -1
    return spins, best_E, best_idx

def tabu_local_search(spins: np.ndarray, iters: int = 150, tabu_len: int = 10):
    """Perform tabu search starting from given spins."""
    s = spins.copy()
    best_s = s.copy()
    best_E = labs_energy(s)

    tabu_queue = deque(maxlen=tabu_len)
    tabu_set = set()

    for _ in range(iters):
        s, E, flipped = best_single_flip(s, tabu_set)
        if flipped is None:
            break

        tabu_queue.append(flipped)
        tabu_set = set(tabu_queue)

        if E < best_E:
            best_E = E
            best_s = s.copy()

    return best_s, best_E


In [ ]:
@dataclass
class MTSConfig:
    """Configuration for Memetic Tabu Search."""
    N: int = 32
    K: int = 30
    steps: int = 150
    p_mut: float = None  # Will be set to 1/N
    local_iters: int = 100
    tabu_len: int = 10
    seed: int = 42

    def __post_init__(self):
        if self.p_mut is None:
            self.p_mut = 1.0 / self.N

def init_population_random(cfg: MTSConfig, rng: np.random.Generator):
    """Initialize population with random bitstrings."""
    pop = []
    energies = []
    for _ in range(cfg.K):
        spins = rng.choice([-1, 1], size=cfg.N).astype(np.int8)
        pop.append(spins)
        energies.append(labs_energy(spins))
    return pop, np.array(energies)

def init_population_from_seeds(cfg: MTSConfig, seed_spins_list: list[np.ndarray], rng: np.random.Generator):
    """
    Initialize population from quantum-generated seeds.
    If fewer than K seeds, fill remainder randomly.
    If more than K, keep best K by energy.
    """
    # Score all seeds
    scored = [(labs_energy(s), s.astype(np.int8)) for s in seed_spins_list]
    scored.sort(key=lambda x: x[0])

    # Take best K seeds, fill remainder randomly if needed
    pop = [s for _, s in scored[:cfg.K]]
    while len(pop) < cfg.K:
        spins = rng.choice([-1, 1], size=cfg.N).astype(np.int8)
        pop.append(spins)

    energies = np.array([labs_energy(s) for s in pop])
    return pop, energies

def run_mts_core(cfg: MTSConfig, init_pop: list[np.ndarray], init_energies: np.ndarray, rng: np.random.Generator):
    """
    Core MTS loop. Accepts pre-initialized population.
    Returns best solution and runtime.
    """
    pop = [s.copy() for s in init_pop]
    energies = init_energies.copy()

    best_idx = int(np.argmin(energies))
    best_s = pop[best_idx].copy()
    best_E = int(energies[best_idx])

    history = [best_E]
    start = time.time()

    for _ in range(cfg.steps):
        i, j = rng.integers(0, cfg.K, size=2)
        child = combine_uniform(pop[i], pop[j], rng)
        child = mutate_flip_bits(child, cfg.p_mut, rng)
        child, Echild = tabu_local_search(child, cfg.local_iters, cfg.tabu_len)

        worst = int(np.argmax(energies))
        if Echild < int(energies[worst]):
            pop[worst] = child
            energies[worst] = Echild
            if Echild < best_E:
                best_E = int(Echild)
                best_s = child.copy()

        history.append(best_E)

    runtime = time.time() - start
    return {
        "best_spins_pm1": best_s,
        "best_energy": best_E,
        "runtime": runtime,
        "history": history
    }


## Quantum Sampling with CUDA-Q

Counterdiabatic circuit implementation for generating quantum-seeded populations.


In [ ]:
# Import utility functions for theta computation
import sys
sys.path.append('tutorial_notebook/auxiliary_files')
import labs_utils as utils

def get_interactions(N: int):
    """
    Generate G2 (two-body) and G4 (four-body) interaction indices.
    Based on Eq. 15 loop structure from the paper.
    """
    G2 = []
    G4 = []

    # Two-body terms
    for i in range(0, N - 2):
        k_max = (N - i - 1) // 2
        for k in range(1, k_max + 1):
            G2.append([i, i + k])

    # Four-body terms
    for i in range(0, N - 3):
        t_max = (N - i - 2) // 2
        for t in range(1, t_max + 1):
            k_max = N - i - t - 1
            for k in range(t + 1, k_max + 1):
                G4.append([i, i + t, i + k, i + k + t])

    return G2, G4


In [ ]:
@cudaq.kernel
def rzz(theta: float, a: cudaq.qubit, b: cudaq.qubit):
    """Two-qubit ZZ rotation."""
    cx(a, b)
    rz(theta, b)
    cx(a, b)

@cudaq.kernel
def rzzzz(theta: float, q0: cudaq.qubit, q1: cudaq.qubit, q2: cudaq.qubit, q3: cudaq.qubit):
    """Four-qubit ZZZZ rotation."""
    cx(q0, q3)
    cx(q1, q3)
    cx(q2, q3)
    rz(theta, q3)
    cx(q2, q3)
    cx(q1, q3)
    cx(q0, q3)

@cudaq.kernel
def trotterized_circuit(N: int,
                        G2: list[list[int]],
                        G4: list[list[int]],
                        steps: int,
                        dt: float,
                        T: float,
                        thetas: list[float]):
    """
    Full trotterized counterdiabatic circuit for LABS.
    Applies all two- and four-body terms per Trotter step.
    """
    reg = cudaq.qvector(N)
    h(reg)  # Initial state: |+>^N

    for s in range(steps):
        th_s = thetas[s]
        ang2 = 4.0 * th_s
        ang4 = 8.0 * th_s

        # Apply two-body terms
        for pair in G2:
            i = pair[0]
            j = pair[1]
            rzz(ang2, reg[i], reg[j])

        # Apply four-body terms
        for quad in G4:
            a = quad[0]
            b = quad[1]
            c = quad[2]
            d = quad[3]
            rzzzz(ang4, reg[a], reg[b], reg[c], reg[d])

    mz(reg)


In [ ]:
def bitstring_to_bits01(bitstr: str, N: int) -> np.ndarray:
    """Convert bitstring to 0/1 array, padding if needed."""
    if len(bitstr) != N:
        bitstr = bitstr.zfill(N)
    return np.array([1 if c == '1' else 0 for c in bitstr], dtype=np.int8)

def counts_to_unique_bitstrings(counts) -> list[str]:
    """Extract unique bitstrings from CUDA-Q sample counts."""
    keys = []
    try:
        for k in counts:
            keys.append(str(k))
    except TypeError:
        # Fallback: parse string representation
        s = str(counts).replace('{', '').replace('}', '').strip()
        if s:
            for token in s.split():
                if ':' in token:
                    keys.append(token.split(':')[0])
    return keys

def quantum_seed_population(N: int, K: int, n_steps: int = 1, T: float = 1.0, shots: int = 300):
    """
    Generate quantum-seeded population using CUDA-Q sampling.
    
    Args:
        N: Sequence length
        K: Target population size
        n_steps: Number of Trotter steps
        T: Total evolution time
        shots: Number of quantum samples
        
    Returns:
        List of spin arrays (±1) for top-K candidates by energy
    """
    dt = T / n_steps
    G2, G4 = get_interactions(N)

    # Precompute theta values for each Trotter step
    thetas = []
    for step in range(1, n_steps + 1):
        t = step * dt
        thetas.append(utils.compute_theta(t, dt, T, N, G2, G4))

    # Sample from quantum circuit
    counts = cudaq.sample(
        trotterized_circuit,
        N, G2, G4, n_steps, dt, T, thetas,
        shots_count=shots
    )

    # Convert sampled bitstrings to spins and score
    candidates = []
    bitstrings = counts_to_unique_bitstrings(counts)
    
    for b in bitstrings:
        bits01 = bitstring_to_bits01(b, N)
        spins = bits01_to_pm1(bits01)
        E = labs_energy(spins)
        candidates.append((E, spins))

    # Sort by energy and return top K
    candidates.sort(key=lambda x: x[0])
    top_seeds = [s for _, s in candidates[:K]]
    
    return top_seeds


## Phase 2 Experiment Loop

Structured experiments for N ∈ {32, 48, 64} with 5 random seeds per configuration.


In [ ]:
# Experiment configuration
SEQUENCE_LENGTHS = [32, 48, 64]
RANDOM_SEEDS = [42, 123, 456, 789, 1024]
POPULATION_SIZE = 30
MTS_STEPS = 150
LOCAL_ITERS = 100
TABU_LEN = 10

# Quantum sampling parameters
QUANTUM_SHOTS = 300
TROTTER_STEPS = 1
EVOLUTION_TIME = 1.0

# Results storage
results = []

print("=" * 60)
print("Phase 2 CPU Scaling Experiments")
print("=" * 60)
print(f"Sequence lengths: {SEQUENCE_LENGTHS}")
print(f"Random seeds: {RANDOM_SEEDS}")
print(f"Population size (K): {POPULATION_SIZE}")
print(f"MTS steps: {MTS_STEPS}")
print("=" * 60)


In [ ]:
# Main experiment loop
for N in SEQUENCE_LENGTHS:
    print(f"\n{'='*60}")
    print(f"Starting experiments for N = {N}")
    print(f"{'='*60}")
    
    for seed_idx, seed in enumerate(RANDOM_SEEDS, 1):
        print(f"\n--- Seed {seed_idx}/5: {seed} ---")
        
        # Configure MTS with current N and seed
        cfg = MTSConfig(
            N=N,
            K=POPULATION_SIZE,
            steps=MTS_STEPS,
            p_mut=1.0 / N,
            local_iters=LOCAL_ITERS,
            tabu_len=TABU_LEN,
            seed=seed
        )
        
        rng = np.random.default_rng(seed)
        
        # === Random-seeded MTS ===
        print("  [1/2] Running random-seeded MTS...", end=" ", flush=True)
        pop_rand, energies_rand = init_population_random(cfg, rng)
        result_rand = run_mts_core(cfg, pop_rand, energies_rand, rng)
        
        print(f"Done. Best energy: {result_rand['best_energy']}, Runtime: {result_rand['runtime']:.2f}s")
        
        results.append({
            "N": N,
            "seed": seed,
            "init_type": "random",
            "best_energy": int(result_rand['best_energy']),
            "runtime_seconds": float(result_rand['runtime'])
        })
        
        # === Quantum-seeded MTS ===
        print("  [2/2] Running quantum-seeded MTS...", end=" ", flush=True)
        
        # Generate quantum seeds (uses CUDA-Q sampling on CPU)
        quantum_seeds = quantum_seed_population(
            N=N,
            K=cfg.K,
            n_steps=TROTTER_STEPS,
            T=EVOLUTION_TIME,
            shots=QUANTUM_SHOTS
        )
        
        # Re-initialize RNG to ensure identical MTS conditions
        rng = np.random.default_rng(seed)
        pop_q, energies_q = init_population_from_seeds(cfg, quantum_seeds, rng)
        result_q = run_mts_core(cfg, pop_q, energies_q, rng)
        
        print(f"Done. Best energy: {result_q['best_energy']}, Runtime: {result_q['runtime']:.2f}s")
        
        results.append({
            "N": N,
            "seed": seed,
            "init_type": "quantum",
            "best_energy": int(result_q['best_energy']),
            "runtime_seconds": float(result_q['runtime'])
        })

print(f"\n{'='*60}")
print("All experiments completed!")
print(f"Total runs: {len(results)}")
print(f"{'='*60}")


## Save Results to JSON

Export results to `results/phase2_cpu_results.json` for analysis.


In [ ]:
# Save results to JSON
output_path = "results/phase2_cpu_results.json"

with open(output_path, 'w') as f:
    json.dump({
        "metadata": {
            "experiment": "Phase 2 CPU Scaling",
            "date": time.strftime("%Y-%m-%d %H:%M:%S"),
            "target": cudaq.get_target().name,
            "sequence_lengths": SEQUENCE_LENGTHS,
            "random_seeds": RANDOM_SEEDS,
            "population_size": POPULATION_SIZE,
            "mts_steps": MTS_STEPS,
            "local_iters": LOCAL_ITERS,
            "tabu_len": TABU_LEN,
            "quantum_shots": QUANTUM_SHOTS,
            "trotter_steps": TROTTER_STEPS,
            "evolution_time": EVOLUTION_TIME
        },
        "results": results
    }, f, indent=2)

print(f"\n✓ Results saved to: {output_path}")
print(f"  Total entries: {len(results)}")


## Results Summary and Visualization


In [ ]:
# Display summary statistics
df = pd.DataFrame(results)

print("\n" + "="*60)
print("RESULTS SUMMARY")
print("="*60)

for N in SEQUENCE_LENGTHS:
    print(f"\nN = {N}:")
    df_n = df[df['N'] == N]
    
    for init_type in ['random', 'quantum']:
        df_type = df_n[df_n['init_type'] == init_type]
        mean_energy = df_type['best_energy'].mean()
        std_energy = df_type['best_energy'].std()
        mean_runtime = df_type['runtime_seconds'].mean()
        
        print(f"  {init_type:8s}: Energy = {mean_energy:.1f} ± {std_energy:.1f}, "
              f"Runtime = {mean_runtime:.2f}s")


In [ ]:
# Visualization: Energy comparison
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for idx, N in enumerate(SEQUENCE_LENGTHS):
    ax = axes[idx]
    df_n = df[df['N'] == N]
    
    random_energies = df_n[df_n['init_type'] == 'random']['best_energy'].values
    quantum_energies = df_n[df_n['init_type'] == 'quantum']['best_energy'].values
    
    x = np.arange(len(RANDOM_SEEDS))
    width = 0.35
    
    ax.bar(x - width/2, random_energies, width, label='Random', alpha=0.8, color='steelblue')
    ax.bar(x + width/2, quantum_energies, width, label='Quantum', alpha=0.8, color='orange')
    
    ax.set_xlabel('Seed Index')
    ax.set_ylabel('Best Energy')
    ax.set_title(f'N = {N}')
    ax.set_xticks(x)
    ax.set_xticklabels([f'{i+1}' for i in range(len(RANDOM_SEEDS))])
    ax.legend()
    ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('results/phase2_cpu_energy_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Visualization saved to: results/phase2_cpu_energy_comparison.png")


In [ ]:
# Runtime comparison
fig, ax = plt.subplots(figsize=(10, 6))

for init_type, color, marker in [('random', 'steelblue', 'o'), ('quantum', 'orange', 's')]:
    df_type = df[df['init_type'] == init_type]
    grouped = df_type.groupby('N')['runtime_seconds'].mean()
    ax.plot(grouped.index, grouped.values, marker=marker, label=init_type, 
            color=color, linewidth=2, markersize=8)

ax.set_xlabel('Sequence Length (N)', fontsize=12)
ax.set_ylabel('Mean Runtime (seconds)', fontsize=12)
ax.set_title('Phase 2 CPU Runtime Comparison', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('results/phase2_cpu_runtime_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Runtime plot saved to: results/phase2_cpu_runtime_comparison.png")


## Phase 2 Complete

All CPU experiments have been completed successfully. Results are saved in:
- `results/phase2_cpu_results.json` (raw data)
- `results/phase2_cpu_energy_comparison.png` (energy visualization)
- `results/phase2_cpu_runtime_comparison.png` (runtime visualization)

**Next Steps:**
- Analyze results to identify trends in quantum vs. random seeding
- Prepare for GPU-accelerated Phase 2 experiments on Brev
- Consider larger N values (e.g., 80, 96) for GPU runs
